<a href="https://colab.research.google.com/github/salomeeja/algo_trading/blob/main/portfolio_optimization_2_asseets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import datetime as dt
import pandas as pd
import numpy as np

In [ ]:
symbols = ['GOOG', 'CAT']
df = yf.download(symbols, start='2005-01-01', end='2015-01-01', interval='1mo')['Adj Close'].dropna()

[*********************100%***********************]  2 of 2 completed


In [ ]:
df

,CAT,GOOG
Date,,
2005-01-01,28.513,97.445
2005-02-01,30.551,93.644
2005-03-01,29.391,89.918
2005-04-01,28.301,109.589
2005-05-01,30.390,138.117
...,...,...
2014-08-01,87.975,570.035
2014-09-01,79.877,575.779
2014-10-01,81.797,557.549


Return columns

In [ ]:
start_price = 100
end_price = 120
profit = end_price / start_price - 1
profit

0.19999999999999996

In [ ]:
def add_retun_columns(symbols):
  pd.options.display.float_format = '{:.3f}'.format
  for symbol in symbols:
    df[f'{symbol}_Ret'] = df[symbol] / df[symbol].shift(1) - 1

In [ ]:
add_retun_columns(symbols)

In [ ]:
df.dropna()

,CAT,GOOG,GOOG_Ret,CAT_Ret
Date,,,,
2005-02-01,30.551,93.644,-0.039,0.071
2005-03-01,29.391,89.918,-0.040,-0.038
2005-04-01,28.301,109.589,0.219,-0.037
2005-05-01,30.390,138.117,0.260,0.074
2005-06-01,30.778,146.526,0.061,0.013
...,...,...,...,...
2014-08-01,87.975,570.035,0.000,0.089
2014-09-01,79.877,575.779,0.010,-0.092
2014-10-01,81.797,557.549,-0.032,0.024


In [ ]:
df = df[['GOOG_Ret', 'CAT_Ret']].dropna()
df.corr()

,GOOG_Ret,CAT_Ret
GOOG_Ret,1.000,0.196
CAT_Ret,0.196,1.000


Standard Deviation

In [ ]:
prices = df['CAT_Ret'][:5]
prices

Date
2005-02-01    0.071
2005-03-01   -0.038
2005-04-01   -0.037
2005-05-01    0.074
2005-06-01    0.013
Name: CAT_Ret, dtype: float64

In [ ]:
prices.std()

0.05513568346107534

In [ ]:
prices_sum = sum(prices)
prices_sum

0.08299048010202625

In [ ]:
mean = prices_sum / 5
mean

0.01659809602040525

In [ ]:
deviation_from_the_mean = prices - mean 
deviation_from_the_mean

Date
2005-02-01    0.055
2005-03-01   -0.055
2005-04-01   -0.054
2005-05-01    0.057
2005-06-01   -0.004
Name: CAT_Ret, dtype: float64

In [ ]:
deviation_squares = deviation_from_the_mean ** 2
deviation_squares

Date
2005-02-01   0.003
2005-03-01   0.003
2005-04-01   0.003
2005-05-01   0.003
2005-06-01   0.000
Name: CAT_Ret, dtype: float64

In [ ]:
sum_of_squares = sum(deviation_squares)
sum_of_squares

0.01215977436287959

In [ ]:
variance =  sum_of_squares / (5 - 1)
variance

0.0030399435907198973

In [ ]:
stdv = np.sqrt(variance)
stdv

0.05513568346107534

Create stat-Board

In [ ]:
def stat_board(symbols):
  columns = [
      'annualized_mean_return',
      'annualized_std',
      'sharp_ratio'
  ]
  board = pd.DataFrame(index=columns)
  for symbol in symbols:
    board[symbol] = [
        df[f'{symbol}_Ret'].mean() * 12,
        df[f'{symbol}_Ret'].std() * np.sqrt(12),
        (df[f'{symbol}_Ret'].mean() * 12) / (df[f'{symbol}_Ret'].std() * np.sqrt(12))
        ]


  return board
stat_board(symbols)

,GOOG,CAT
annualized_mean_return,0.220,0.155
annualized_std,0.319,0.337
sharp_ratio,0.690,0.460


Create Portfolio board


In [ ]:
board = stat_board(symbols)


w = [0.5, 0.5]
mean = board[symbols[0]].annualized_mean_return * w[0] + board[symbols[1]].annualized_mean_return * w[1]
cov = df[f'{symbols[0]}_Ret'].cov(df[f'{symbols[1]}_Ret'])
std = np.sqrt(
    w[0]**2 *board[symbols[0]].annualized_std**2 +
    w[1]**2 *board[symbols[1]].annualized_std**2 +
    2 * w[0] * w[1] * cov
)
sr = mean / std 
sr

def portfolio_board(symbols):
  print('Portfolio Weights: \n', w)
  columns = [
      'portfolio_annualized_mean',
      'portfolio_annualized_std',
      'portfolio_sharp_ratio'
  ]
  portfolio_board = pd.DataFrame(index=columns) 
  portfolio_board['Portfolio Stat'] = [
      mean,
      std,
      sr
  ]
  print(portfolio_board)

std

0.23363153406704074

In [ ]:
portfolio_board(symbols)

Portfolio Weights: 
 [0.5, 0.5]
                           Portfolio Stat
portfolio_annualized_mean           0.187
portfolio_annualized_std            0.234
portfolio_sharp_ratio               0.802
